In [ ]:
import gradio as gr
import pandas as pd
import requests

# Load transaction dataset
df = pd.read_csv("/kaggle/input/data-ai/jordan_transactions.csv")

API_KEY = "priv"
API_URL = "priv"
MODEL = "gpt-4o-mini"

chat_history = []

# Function to convert DataFrame head to string for context
def get_data_sample_string(df):
    return df.head(1753).to_string(index=False)

# Function to build prompt and call OpenAI
def ask_gpt_with_data(user_input, history):
    history.append({"role": "user", "content": user_input})
    
    # Add example data to the context
    system_prompt = (
        "You are a financial data assistant. Answer the user's questions using the following sample retail transaction data:\n\n"
        + get_data_sample_string(df) +
        "\n\nRespond clearly and concisely. If the answer requires full data analysis, mention that only a sample is shown."
    )

    payload = {
        "model": MODEL,
        "messages": [{"role": "system", "content": system_prompt}] + history,
        "stream": False
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    
    if response.status_code == 200:
        reply = response.json()["choices"][0]["message"]["content"]
        history.append({"role": "assistant", "content": reply})
        messages = [(msg["content"], history[i+1]["content"]) for i, msg in enumerate(history[:-1]) if msg["role"] == "user"]
        return messages, history
    else:
        return [("Error", response.text)], history

with gr.Blocks(title="Jordan Mall Transactions Chatbot") as demo:
    gr.Markdown("## 💬 Jordan Mall Financial Assistant\nAsk anything about transaction patterns, failures, or mall performance.")
    chatbot = gr.Chatbot()
    state = gr.State([])
    txt = gr.Textbox(show_label=False, placeholder="Ask about failed transactions, top malls, etc...", container=False)

    txt.submit(ask_gpt_with_data, [txt, state], [chatbot, state])
    txt.submit(lambda: "", None, txt)

demo.launch()